In [1]:
from surface import grammar;
from surface import converter
from collections import defaultdict
import ast

In [2]:
def get_parse(fn, conll):
    with open(fn, "r") as f:
        next(f)
        conll_parse = {}
        parse = next(f).strip()
        text = [n.strip() for n in parse.strip("[]").split(",")]
        text_parse = []
        for i, w_id in enumerate(text):
            conll_parse[i] = conll[w_id.split("_")[1]]
            text_parse.append(conll[w_id.split("_")[1]][1])
        return text_parse, conll_parse

In [3]:
def set_parse(fn, graph):
    with open(fn, "w+") as f:
        f.write("# IRTG unannotated corpus file, v1.0\n")
        f.write("# interpretation ud: de.up.ling.irtg.algebra.graph.GraphAlgebra\n")
        f.write(graph + "\n")
        f.write("(dummy_0 / dummy_0)\n")

In [4]:
grammar.train("/home/adaamko/data/UD-train/en_ewt-ud-train.conllu", "/home/adaamko/data/T1-dev/en_ewt-ud-dev.conllu")

In [5]:
GRAMMAR_FILE = "train_grammar"
TERMINAL_FILE_ORIG = "../en_ewt-ud-dev-orig.conllu"
TERMINAL_FILE_MIXED = "../en_ewt-ud-dev-mixed.conllu"

In [7]:
rules, _ = converter.extract_rules(TERMINAL_FILE_MIXED)
graphs, _, id_graphs= converter.convert(TERMINAL_FILE_ORIG)
_, sentences, _ = converter.convert(TERMINAL_FILE_ORIG)
conll = grammar.get_conll_from_file(TERMINAL_FILE_ORIG)

In [ ]:
id_to_parse = {}
for i in range(len(rules)):    
    grammar_fn = open('dep_grammar_spec.irtg', 'w') 
    grammar.generate_grammar(GRAMMAR_FILE, rules[i], grammar_fn)
    grammar.generate_terminals(TERMINAL_FILE_MIXED, grammar_fn)
    grammar_fn.close()
    set_parse("ewt_ones", graphs[i])
    !java -Xmx32G -cp alto-2.3.6-SNAPSHOT-all.jar de.up.ling.irtg.script.ParsingEvaluator ewt_ones -g dep_grammar_spec.irtg -I ud -O string=toString -o surface_eval_ewt
    parse = get_parse("surface_eval_ewt")
    id_to_parse[i] = parse

In [13]:
with open("result_parse", "w+") as f:
    for i in id_to_parse:
        f.write(sentences[i] + "\n")
        f.write(graphs[i] + "\n")
        f.write(str(id_to_parse[i]) + "\n")
        f.write("\n")

In [8]:
sen_id = 1623
grammar_fn = open('dep_grammar_spec.irtg', 'w') 
grammar.generate_grammar(GRAMMAR_FILE, rules[sen_id], grammar_fn)
grammar.generate_terminal_ids(conll[sen_id], grammar_fn)
grammar_fn.close()
set_parse("ewt_ones", id_graphs[sen_id])
!java -Xmx32G -cp alto-2.3.6-SNAPSHOT-all.jar de.up.ling.irtg.script.ParsingEvaluator ewt_ones -g dep_grammar_spec.irtg -I ud -O string=toString -o surface_eval_ewt
text_parse, conll_parse = get_parse("surface_eval_ewt", conll[sen_id])

Processing ewt_ones (2 instances) ...
1 [[VERB_8/VERB_8 -advmod-> ADV_7/ADV_7; VERB_8 -aux-] 424 ms
2 [[dummy_0/dummy_0]                                 ] 3 ms
Done, total time: 489 ms


In [9]:
sentences[1623]

'# text = Cécile is a hairdresser and has just moved into the neighbourhood.'

In [ ]:
print(text_parse)
print(conll_parse)

In [ ]:
graphs[278]

In [ ]:
id_graphs[1048]

In [ ]:
conll[1048]